# ~ Rhythm Shah

~ 1233960561

~ 03/04/2025

## Code Cell 1 (5%) - Import Required Libraries and Load Data

### Import all necessary libraries for data processing, NLP, and interaction with LLMs.

In [136]:
import numpy as numpy
import pandas as pd
import s3fs
import json
import boto3
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

### Load restaurant_reviews_az.csv.

In [137]:
df = pd.read_csv('restaurant_reviews_az.csv')[:2000]

In [138]:
df.shape

(2000, 9)

## Code Cell 2 (5%) - Data Preprocessing

### Remove 3-star reviews from the dataset.

In [139]:
df = df[df['stars'] != 3]

### Create a new column Sentiment where:
Reviews with 1 or 2 stars are labeled as 0 (Negative Sentiment).
Reviews with 4 or 5 stars are labeled as 1 (Positive Sentiment).

In [140]:
df['Sentiment'] = df['stars'].apply(lambda x: 1 if x >= 4 else 0)

### Create a dataset for this assignment by randomly selecting 50 positive reviews and 50 negative reviews

In [141]:
positive_reviews = df[df['Sentiment'] == 1].sample(n=50)
negative_reviews = df[df['Sentiment'] == 0].sample(n=50)

df1 = pd.concat([positive_reviews, negative_reviews]).reset_index(drop=True)
df1.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,Sentiment
0,YbHV-lgpUp61VmYAPzqboQ,ZYnuyFg0cOEI9dEbYEwXjw,UCMSWPqzXjd7QHq7v8PJjQ,5,1,0,0,Great food & wonderful service. Have dined ma...,2021-07-18 04:49:48,1
1,CIfMJeySdInsnfZqBcQMKw,5oQ8GodXqXOtDmFs3vF2JQ,f82dhKNiUXsDVPMLqKYiIQ,5,0,0,0,I love this place. I love the renovations. I...,2020-02-15 16:29:48,1
2,wZlBimICAi-l1D7ln9Mjeg,TX2lJ9S0GskrZ6X9bNodyg,wSeEtJ-d2C8vqy5v9La9-g,4,0,0,0,We came to vail to see the Sahuaro buttes and ...,2021-06-07 00:38:50,1
3,zl28SdOchQyQLecngKuGgA,FvuREyQJqxnTS3Oy731AIA,OMnPtRGmbY8qH_wIILfYKA,5,3,1,2,Their bruschetta are really good but the porti...,2021-03-14 17:54:33,1
4,_zp_f2Hky8NcC8LgZrGd2w,mMJ5EPzkwAkI87bM7_RzNA,OMnPtRGmbY8qH_wIILfYKA,4,0,0,0,Soooooo gooood. They have their daily special...,2020-10-04 03:20:54,1


In [142]:
df1.shape

(100, 10)

## Code Cell 3 (20%) - Perform Sentiment Analysis Using Zero-Shot Learning

### Use a Claude 3 Sonnet for zero-shot prompting.

In [143]:
client = boto3.client("bedrock-runtime", region_name = "us-east-1") 
model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'

### Predict sentiment labels for the selected 100 reviews without providing any labeled training examples.

In [144]:
def classify_sentiments(data):
    results = []

    for i in data:
        messages = [{"role": "user", "content": f"In only one word classify the sentiment of '{data}' into either Positive or Negative only. Do not use any other words for classification. Use only Positive or Negative"}]

        request_body = {"anthropic_version": "bedrock-2023-05-31", "messages": messages, "max_tokens": 512, "temperature": 0.1, "top_p": 1.0}

        try:
            response = client.invoke_model(modelId = model_id, contentType = 'application/json', body = json.dumps(request_body))
            result = json.loads(response['body'].read().decode())
            sentiment = result['content'][0]['text'].strip()
            results.append(sentiment)
        except Exception as e:
            print(f"Error processing text '{i}': {e}")
            results.append("Error")
    return results

In [145]:
df1['Claude3_Prediction'] = classify_sentiments(df1['text'])

In [146]:
df1.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,Sentiment,Claude3_Prediction
0,YbHV-lgpUp61VmYAPzqboQ,ZYnuyFg0cOEI9dEbYEwXjw,UCMSWPqzXjd7QHq7v8PJjQ,5,1,0,0,Great food & wonderful service. Have dined ma...,2021-07-18 04:49:48,1,Positive
1,CIfMJeySdInsnfZqBcQMKw,5oQ8GodXqXOtDmFs3vF2JQ,f82dhKNiUXsDVPMLqKYiIQ,5,0,0,0,I love this place. I love the renovations. I...,2020-02-15 16:29:48,1,Positive
2,wZlBimICAi-l1D7ln9Mjeg,TX2lJ9S0GskrZ6X9bNodyg,wSeEtJ-d2C8vqy5v9La9-g,4,0,0,0,We came to vail to see the Sahuaro buttes and ...,2021-06-07 00:38:50,1,Positive
3,zl28SdOchQyQLecngKuGgA,FvuREyQJqxnTS3Oy731AIA,OMnPtRGmbY8qH_wIILfYKA,5,3,1,2,Their bruschetta are really good but the porti...,2021-03-14 17:54:33,1,Positive
4,_zp_f2Hky8NcC8LgZrGd2w,mMJ5EPzkwAkI87bM7_RzNA,OMnPtRGmbY8qH_wIILfYKA,4,0,0,0,Soooooo gooood. They have their daily special...,2020-10-04 03:20:54,1,Positive


In [147]:
df1['Claude3_Prediction'].value_counts()

Claude3_Prediction
Positive    100
Name: count, dtype: int64

### Evaluate model performance using precision, recall, f1, and accuracy.

In [155]:
def evaluate_model_performance(df, pred_col, true_col = "Sentiment"):
    y_true = df[true_col]
    y_pred = df[pred_col].map({"Positive": 1, "Negative": 0})
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")

In [149]:
evaluate_model_performance(df1, "Claude3_Prediction")

Accuracy: 50.00%
Precision: 0.50
Recall: 1.00
F1-Score: 0.67


## Code Cell 4 (20%) - Perform Sentiment Analysis Using Few-Shot Learning

### Select a few examples for few-shot learning

In [150]:
positive_reviews = df[df['Sentiment'] == 1].sample(n=50)
negative_reviews = df[df['Sentiment'] == 0].sample(n=50)

df2 = pd.concat([positive_reviews, negative_reviews]).reset_index(drop=True)
df2.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,Sentiment
0,syOamw-s0vm47Va68rN04Q,4BS4PMY8JCiV09mrgaEmpA,B10mqANgHvL8gBteo1UhAA,4,0,0,0,This restaurant has an authentic Italian vibe ...,2021-05-09 09:48:15,1
1,xsAs_TwvQWCKqoO47s3pcw,l4qBjoAGw9qvfVWT6kRBIA,fgTOJRkc703E4XRdcr5zRA,4,0,0,0,I will say that the drinks do taste more real ...,2020-08-03 00:41:48,1
2,By1aYbG4pTW3_WK0wKMfrg,qsJJKbrm4ce_0MUFRpox6A,8QJTzIaR5FceGds8HBSIig,5,0,0,0,Barrio Brewing is a great place to go. Don't b...,2020-10-11 01:58:27,1
3,_bUyjGDqeJst7nDih6hAHw,rqzcmK4ysvFA_sbgYaSmdg,UCMSWPqzXjd7QHq7v8PJjQ,5,4,0,1,I am a big breakfast and brunch eater when I v...,2020-01-11 14:42:29,1
4,HO2GaavIMgnYnsJcdwLIVw,NbDPSGB_lfZXOwOcAJX3oA,OMnPtRGmbY8qH_wIILfYKA,5,0,0,0,I'm in love! The wine glasses are crystal clea...,2021-01-30 19:04:23,1


### Use the example(s) to guide the LLM in classifying sentiment for the selected 100 reviews.

In [151]:
few_shot_prefix = """
Examples:
1. "Amazing food, great service!" -> Positive
2. "Terrible experience, rude staff." -> Negative
3. "It was okay, nothing special." -> Positive
4. "Slow delivery, cold meal." -> Negative
"""
df2['Claude3_FewShot_Prediction'] = classify_sentiments(few_shot_prefix + df2['text'])

In [152]:
df2.Claude3_FewShot_Prediction.value_counts()

Claude3_FewShot_Prediction
Negative    97
Positive     3
Name: count, dtype: int64

### Evaluate model performance using precision, recall, f1, and accuracy.

In [156]:
evaluate_model_performance(df2, "Claude3_FewShot_Prediction")

Accuracy: 53.00%
Precision: 1.00
Recall: 0.06
F1-Score: 0.11


## Code Cell 5 (20%) - Experiment with Multiple LLMs

### Select two distinct LLMs (e.g., Claude, LLaMA) for sentiment analysis.